#4-3 サポートベクトルマシン

## 概要

**サポートベクトルマシンは、データ集合を分類するための手法です。**

サポートベクトルマシンでは、データを分類するための境界線とデータの最短距離をマージンとし、このマージンを最大化することで、値を分類するのに良い決定境界線を求めます。
マージンとは、学習データのうち最も決定境界線に近いものと、決定境界線との距離です。　※下記はマージン最大化のイメージ

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_03_01.jpg?raw=true" width="320px">

マージンの内側にデータが入ることを許容しないことをハードマージンと呼びます。

一部のデータがマージンの内側に入ることを許容することをソフトマージンと呼びます。ソフトマージンで、一部の誤分類を寛容にするためにスラック変数と呼ばれる変数を用います。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_03_02.jpg?raw=true" width="480px">

サポートベクトルマシンには、決定境界が線形の線形サポートベクトルマシン、
決定境界が非線型のカーネル法のサポートベクトルマシンがあります。カーネル法
で、高速に計算するために、計算量を大幅に削減する方法をカーネルトリックと呼びます。

サポートベクトルマシン（SVM）は、線形／非線形分類、回帰だけでなく、外れ値検出さえできる非常に強力で柔軟な機械学習モデルであります。機械学習でもっとも人気のあるモデルのひとつでありSVM は複雑ながら中小規模のデータセットの分類に特に適しています。

## アヤメデータを用いた実装

環境設定を行います

In [0]:
# Python 2, 3 をサポートします
from __future__ import division, print_function, unicode_literals

# 標準ライブラリのインポート
import numpy as np
import os

# 乱数の固定
np.random.seed(42)

# プロットの設定
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 保存先ディレクトリの設定
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "svm"
IMAGE_DIR = "images"

os.makedirs(os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR, CHAPTER_ID), exist_ok=True)

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR , CHAPTER_ID, fig_id)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(image_path(fig_id) + ".png", format='png', dpi=300)

iris (あやめ) データセットを使ってSVMをじっそうしてみます。

iris（あやめ）は、セトサ（Iris-Setosa）、バージカラー（Iris-Versicolor）、バージニカ（Iris-Virginica）の3 種類のあやめのがく片（sepal）と花弁（petal）の幅と長さが収められたデータセットです。(前段で説明)

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_06_03.jpg?raw=true" width="460px">

###マージンの大きい分類

In [0]:
from sklearn.svm import SVC
from sklearn import datasets

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

setosa_or_versicolor = (y == 0) | (y == 1)
X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

# SVM Classifier model
svm_clf = SVC(kernel="linear", C=float("inf"))
svm_clf.fit(X, y)

In [0]:
x0 = np.linspace(0, 5.5, 200)
pred_1 = 5*x0 - 20
pred_2 = x0 - 1.8
pred_3 = 0.1 * x0 + 0.5

def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    #  w0*x0 + w1*x1 + b = 0の決定境界で
    # => x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]

    margin = 1/w[1]
    gutter_up = decision_boundary + margin
    gutter_down = decision_boundary - margin

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='#FFAAAA')
    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, gutter_up, "k--", linewidth=2)
    plt.plot(x0, gutter_down, "k--", linewidth=2)

plt.figure(figsize=(12,2.7))

plt.subplot(121)
plt.plot(x0, pred_1, "g--", linewidth=2)
plt.plot(x0, pred_2, "m-", linewidth=2)
plt.plot(x0, pred_3, "r-", linewidth=2)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.axis([0, 5.5, 0, 2])

plt.subplot(122)
plot_svc_decision_boundary(svm_clf, 0, 5.5)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo")
plt.xlabel("Petal length", fontsize=14)
plt.axis([0, 5.5, 0, 2])

save_fig("large_margin_classification_plot")
plt.show()

上の図はiris データセットの一部を示している。このふたつのクラスは、直線で簡単に分割できます（線形分割可能、lineary separable である）。

左のグラフは、考え得る3 種類の線形分類器の決定境界を示しています。破線の決定境界を持つモデルは非常に性能が低く、クラスを正しく分割することさえできていません。ほかのふたつは、この訓練セットに対しては完璧に機能するが、決定境界がインスタンスに近いため、新しいインスタンスに対しても同じような性能を発揮することはできないだでしょう。

それに対し、右側のグラフの実線は、SVM 分類器の決定境界を示しています。この線はふたつのクラスを分割できているだけでなく、もっとも近い訓練インスタンスからの距離ができる限り遠くなるようにしています。SVM 分類器は、クラスの間にできる限り太い道（2 本の平行な破線で表されている）を通すものだと考えることができる。これをマージンの大きい分類と呼びます。
「道から外れた」訓練インスタンスを増やしても、決定境界に影響は及ばないことに注意しましょう。
決定境界は、道の際にあるインスタンスによって決まります（サポートされる）。このようなインスタンスのことをサポートベクトル（support vector）と呼びます（右図で 大きな丸で描かれているもの）。

###特徴量のスケールから影響を受けやすいSVM

In [0]:
Xs = np.array([[1, 50], [5, 20], [3, 80], [5, 60]]).astype(np.float64)
ys = np.array([0, 0, 1, 1])
svm_clf = SVC(kernel="linear", C=100)
svm_clf.fit(Xs, ys)

plt.figure(figsize=(12,3.2))
plt.subplot(121)
plt.plot(Xs[:, 0][ys==1], Xs[:, 1][ys==1], "bo")
plt.plot(Xs[:, 0][ys==0], Xs[:, 1][ys==0], "ms")
plot_svc_decision_boundary(svm_clf, 0, 6)
plt.xlabel("$x_0$", fontsize=20)
plt.ylabel("$x_1$  ", fontsize=20, rotation=0)
plt.title("Unscaled", fontsize=16)
plt.axis([0, 6, 0, 90])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(Xs)
svm_clf.fit(X_scaled, ys)

plt.subplot(122)
plt.plot(X_scaled[:, 0][ys==1], X_scaled[:, 1][ys==1], "bo")
plt.plot(X_scaled[:, 0][ys==0], X_scaled[:, 1][ys==0], "ms")
plot_svc_decision_boundary(svm_clf, -2, 2)
plt.xlabel("$x_0$", fontsize=20)
plt.title("Scaled", fontsize=16)
plt.axis([-2, 2, -2, 2])

save_fig("sensitivity_to_feature_scales_plot")

上図からSVM は、特徴量のスケールの影響を受けやすくあります。左側のグラフでは、縦方向のスケールが横方向のスケールよりもかなり大きいため、可能な道のなかでもっとも太いものはほとんど真横に向かうものになっています。特徴量をスケーリング（たとえば、
scikit-learn のStandardScaler で）したあとの決定境界（右側のグラフ）は、はるかによい感じに見えます。

###ソフトマージン分類

すべてのインスタンスが道に引っかからず、正しい側にいることを厳密に要求する場合、それをハードマージン分類（hard margine classification）と呼びます。ハードマージン分類には、データが
線形分割できるときでなければ使えず、外れ値に敏感になり過ぎるというふたつの大きな問題点があります。下図は、iris データセットに1 個の外れ値を追加したものを示しています。

左側のグラフは、ハードマージンを見つけられないもの、右側のグラフは、外れ値のない最初に示した図とは決定境界がまったく異なり、おそらく同じようには汎化できないものであります。
これらの問題を避けるために、もっと柔軟性の高いモデルを使った方がよいと考えられます。
目標は、道をできる限り太くすることと、マージン違反（margin violation = 道のなかや間違った側に入ってしまう
インスタンス）を減らすこととの間でバランスを取ることであります。これをソフトマージン分類と呼びます。

In [0]:
X_outliers = np.array([[3.4, 1.3], [3.2, 0.8]])
y_outliers = np.array([0, 0])
Xo1 = np.concatenate([X, X_outliers[:1]], axis=0)
yo1 = np.concatenate([y, y_outliers[:1]], axis=0)
Xo2 = np.concatenate([X, X_outliers[1:]], axis=0)
yo2 = np.concatenate([y, y_outliers[1:]], axis=0)

svm_clf2 = SVC(kernel="linear", C=10**9)
svm_clf2.fit(Xo2, yo2)

plt.figure(figsize=(12,2.7))

plt.subplot(121)
plt.plot(Xo1[:, 0][yo1==1], Xo1[:, 1][yo1==1], "bs")
plt.plot(Xo1[:, 0][yo1==0], Xo1[:, 1][yo1==0], "yo")
plt.text(0.3, 1.0, "Impossible!", fontsize=24, color="red")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.annotate("Outlier",
             xy=(X_outliers[0][0], X_outliers[0][1]),
             xytext=(2.5, 1.7),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.1),
             fontsize=16,
            )
plt.axis([0, 5.5, 0, 2])

plt.subplot(122)
plt.plot(Xo2[:, 0][yo2==1], Xo2[:, 1][yo2==1], "bs")
plt.plot(Xo2[:, 0][yo2==0], Xo2[:, 1][yo2==0], "yo")
plot_svc_decision_boundary(svm_clf2, 0, 5.5)
plt.xlabel("Petal length", fontsize=14)
plt.annotate("Outlier",
             xy=(X_outliers[1][0], X_outliers[1][1]),
             xytext=(3.2, 0.08),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.1),
             fontsize=16,
            )
plt.axis([0, 5.5, 0, 2])

save_fig("sensitivity_to_outliers_plot")
plt.show()

scikit-learn のSVM クラス群では、C ハイパーパラメータでこのバランスを調整できます。C が小さければ小さいほど道は太くなるが、マージン違反も増えます。
下図は、線形分割できないデータセットに対して訓練したふたつのソフトマージンSVM 分類器の決定境界とマージンを示しています。左側のグラフは、C として大きな値を使った分類器で、マージン違反は少ないが、マージンが狭くなっています。

右側のグラフは、C として小さな値を使った分類器で、マージンはかなり広いが、道に入り込んでいるインスタンスがかなり多きくなっています。しかし、第2 の分類器の方が汎化性能はよさそうに見えます。実際、この訓練セットでも、第2 の分類器の方が予測誤差は小さくあります。それは、ほとんどのマージン違反が実際には決定境界の正しい側に分類されるからであります。

In [0]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris-Virginica

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X, y)

In [0]:
svm_clf.predict([[5.5, 1.7]])

In [0]:
scaler = StandardScaler()
svm_clf1 = LinearSVC(C=1, loss="hinge", random_state=42)
svm_clf2 = LinearSVC(C=100, loss="hinge", random_state=42)

scaled_svm_clf1 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf1),
    ])
scaled_svm_clf2 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf2),
    ])

scaled_svm_clf1.fit(X, y)
scaled_svm_clf2.fit(X, y)

In [0]:
# スケーリングされていないパラメーターに変換する
b1 = svm_clf1.decision_function([-scaler.mean_ / scaler.scale_])
b2 = svm_clf2.decision_function([-scaler.mean_ / scaler.scale_])
w1 = svm_clf1.coef_[0] / scaler.scale_
w2 = svm_clf2.coef_[0] / scaler.scale_
svm_clf1.intercept_ = np.array([b1])
svm_clf2.intercept_ = np.array([b2])
svm_clf1.coef_ = np.array([w1])
svm_clf2.coef_ = np.array([w2])

# サポートベクトルの検索
t = y * 2 - 1
support_vectors_idx1 = (t * (X.dot(w1) + b1) < 1).ravel()
support_vectors_idx2 = (t * (X.dot(w2) + b2) < 1).ravel()
svm_clf1.support_vectors_ = X[support_vectors_idx1]
svm_clf2.support_vectors_ = X[support_vectors_idx2]

In [0]:
plt.figure(figsize=(12,3.2))
plt.subplot(121)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^", label="Iris-Virginica")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs", label="Iris-Versicolor")
plot_svc_decision_boundary(svm_clf1, 4, 6)
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.title("$C = {}$".format(svm_clf1.C), fontsize=16)
plt.axis([4, 6, 0.8, 2.8])

plt.subplot(122)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
plot_svc_decision_boundary(svm_clf2, 4, 6)
plt.xlabel("Petal length", fontsize=14)
plt.title("$C = {}$".format(svm_clf2.C), fontsize=16)
plt.axis([4, 6, 0.8, 2.8])

save_fig("regularization_plot")

scikit-learn のSVM クラス群では、C ハイパーパラメータでこのバランスを調整できます。C が小さければ小さいほど道は太くなるが、マージン違反も増えます。

上図 は、線形分割できないデータセットに対して訓練したふたつのソフトマージンSVM 分類器の決定境界とマージンを示しています。左側のグラフは、C として大きな値を使った分類器で、マージン違反は少ないが、マージンが狭くなっています。右側のグラフは、C として小さな値を使った分類器で、マージンはかなり広いが、道に入り込んでいるインスタンスがかなり多くなります。しかし、第2 の分類器の方が汎化性能はよさそうに見えます。実際、この訓練セットでも、第2 の分類器の方が予測誤差は小さくなります。それは、ほとんどのマージン違反が実際には決定境界の正しい側に分類されるからであります。

##非線形SVM分類器

線形SVM 分類器は効率的で多くの条件で驚くほどすばらしく機能するが、多くのデータセットは線形分割などとてもできません。このような非線形データセットを処理するためのアプローチのひとつは、多項式特徴量のように特徴量を追加するというものであります。実際、これで線形分割可能なデータセットが得られる場合があります。

下図の左側のグラフを見てみましょう。これは、特徴量がひとつ（名前はx1）の単純なデータセットを表しているが、この単純なデータセットは線形分割不能であります。しかし、x2 = (x1)2 という第2 の特徴量を追加して得られた2 次元データセットは、完全に線形分割可能であります。

In [0]:
X1D = np.linspace(-4, 4, 9).reshape(-1, 1)
X2D = np.c_[X1D, X1D**2]
y = np.array([0, 0, 1, 1, 1, 1, 1, 0, 0])

plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.grid(True, which='both')
plt.axhline(y=0, color='k')
plt.plot(X1D[:, 0][y==0], np.zeros(4), "bs")
plt.plot(X1D[:, 0][y==1], np.zeros(5), "g^")
plt.gca().get_yaxis().set_ticks([])
plt.xlabel(r"$x_1$", fontsize=20)
plt.axis([-4.5, 4.5, -0.2, 0.2])

plt.subplot(122)
plt.grid(True, which='both')
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.plot(X2D[:, 0][y==0], X2D[:, 1][y==0], "bs")
plt.plot(X2D[:, 0][y==1], X2D[:, 1][y==1], "g^")
plt.xlabel(r"$x_1$", fontsize=20)
plt.ylabel(r"$x_2$", fontsize=20, rotation=0)
plt.gca().get_yaxis().set_ticks([0, 4, 8, 12, 16])
plt.plot([-4.5, 4.5], [6.5, 6.5], "r--", linewidth=3)
plt.axis([-4.5, 4.5, -1, 17])

plt.subplots_adjust(right=1)

save_fig("higher_dimensions_plot", tight_layout=False)
plt.show()

scikit-learn を使ってこの考え方を実装するには、PolynomialFeatures 変換器とそのあとにStandardScaler、LinearSVC を組み込んだPipeline を作るとよいです。これをmoons データセットでテストしてみましょう。

In [0]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

def plot_dataset(X, y, axes):
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$", fontsize=20)
    plt.ylabel(r"$x_2$", fontsize=20, rotation=0)

plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.show()

In [0]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X, y)

In [0]:
def plot_predictions(clf, axes):
    x0s = np.linspace(axes[0], axes[1], 100)
    x1s = np.linspace(axes[2], axes[3], 100)
    x0, x1 = np.meshgrid(x0s, x1s)
    X = np.c_[x0.ravel(), x1.ravel()]
    y_pred = clf.predict(X).reshape(x0.shape)
    y_decision = clf.decision_function(X).reshape(x0.shape)
    plt.contourf(x0, x1, y_pred, cmap=plt.cm.brg, alpha=0.2)
    plt.contourf(x0, x1, y_decision, cmap=plt.cm.brg, alpha=0.1)

plot_predictions(polynomial_svm_clf, [-1.5, 2.5, -1, 1.5])
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])

save_fig("moons_polynomial_svc_plot")
plt.show()

###多項式カーネル

多項式特徴量を追加するのは実装が単純であり、あらゆる種類の機械学習アルゴリズム（SVMに限らず）ですばらしく機能するが、次数が低いと非常に複雑なデータセットを処理できず、次数
が高いと特徴量が膨大な数になってモデルが遅くなりすぎます。

しかし、SVM を使う場合は、カーネルトリック（kernel trick）というテクニックを使うことができます。これを使うと、実際に特徴量を追加せずにまるで多くの多項式特徴量を追加したかのような結果が得られます。実際に特徴量を追加するわけではないため、特徴量数の組合せ爆発も生じません。このトリックは、SVC クラスによって実装されています。
moons データセットでテストしてみましょう。

In [0]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)

In [0]:
poly100_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=10, coef0=100, C=5))
    ])
poly100_kernel_svm_clf.fit(X, y)

In [0]:
plt.figure(figsize=(11, 4))

plt.subplot(121)
plot_predictions(poly_kernel_svm_clf, [-1.5, 2.5, -1, 1.5])
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.title(r"$d=3, r=1, C=5$", fontsize=18)

plt.subplot(122)
plot_predictions(poly100_kernel_svm_clf, [-1.5, 2.5, -1, 1.5])
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.title(r"$d=10, r=100, C=5$", fontsize=18)

save_fig("moons_kernelized_polynomial_svc_plot")
plt.show()

上図は3 次元多項式カーネルでSVM 分類器を訓練した結果です。左側のグラフは、この分類器を表しています。右側のグラフには、10 次元多項式カーネルを使った別のSVM 分類器を示しています。

モデルは過学習を起こしており、過学習している場合は、多項回帰モデルの次数を下げなければなりません。逆に、モデルが過小適合しているなら、多項回帰モデルの次数を上げることになります。ハイパーパラメータのcoef0 で、高次多項式モデルと低次多項式モデルからどの程度の影響を認めるかを調節できます。

###ガウスRBF カーネル

多項式特徴量の方式と同様に、類似性特徴量の方式はどの機械学習アルゴリズムでも使えるが、特に訓練セットが大きい場合には、すべての追加特徴量を計算していると、計算量という面でコス
トが高くなってしまう場合があります。

しかし、SVM では、ここでもカーネルトリックが威力を発揮する。実際に類似性特徴量を追加しなくても、多数の類似性特徴量を追加したのと同じ結果が得られるのであります。SVC クラスでガウスRBF カーネルを試してみましょう。

In [0]:
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

In [0]:
from sklearn.svm import SVC

gamma1, gamma2 = 0.1, 5
C1, C2 = 0.001, 1000
hyperparams = (gamma1, C1), (gamma1, C2), (gamma2, C1), (gamma2, C2)

svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_svm_clf = Pipeline([
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
        ])
    rbf_kernel_svm_clf.fit(X, y)
    svm_clfs.append(rbf_kernel_svm_clf)

plt.figure(figsize=(11, 7))

for i, svm_clf in enumerate(svm_clfs):
    plt.subplot(221 + i)
    plot_predictions(svm_clf, [-1.5, 2.5, -1, 1.5])
    plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
    gamma, C = hyperparams[i]
    plt.title(r"$\gamma = {}, C = {}$".format(gamma, C), fontsize=16)

save_fig("moons_rbf_svc_plot")
plt.show()

ガウスRBF カーネルモデルは、左下に示してある。ほかのグラフは、ハイパーパラメータのgammaとC の値を変えて訓練したモデルであります。gamma を増やすと、ベル型の曲線が狭くなり、その結果、各インスタンスの影響を受ける範囲が小さくなります。決定境界
は不規則になり、各インスタンスの周囲でくねくねと曲がります。

逆に、gamma を小さくすると、ベル形の曲線の幅が広くなり、各インスタンスの影響を受ける範囲が広がり、決定境界は滑らかになります。つまり、 は正則化ハイパーパラメータと同じように機能します。モデルが過学習しているときにはgammma を小さくし、過小適合しているときには を大きくするとよいといえます（同じことがC ハイパーパ
ラメータにも当てはまる）。

カーネルはほかにもあるが、RBF カーネルと比べてごくまれにしか使われません。たとえば、特定のデータ構造に専門特化したカーネルがあります。テキストやDNA シーケンスの分類では、文字列カーネル（string kernel）が使われることがあります（たとえば、String Subsequence Kernel やレーベンシュタイン距離：Levenshtein distance に基づくカーネル）。

選べるカーネルがたくさんあるなかで、どれを選んだらよいか、目安として、特に訓練セットが非常に大きい場合や特徴量がたくさんあるときには、まず線形カーネルを選ぶようにしましょう（SVC(kernel="linear") よりもLinearSVC の方がはるかに高速だとい
うことを覚えておきたい）。

訓練セットがそれほど大きくないときには、ガウスRBF カーネルも試してみてもよいです。ほとんどの場合はうまく機能します。そして、時間と計算能力に余裕があり、特に訓練セットのデータ構造に対する専門のカーネルがある場合には、交差検証とグリッドサーチを使ってほかのカーネルを試してみてもよいです。

##(参考)SVM回帰

SVM アルゴリズムは柔軟性が高く、線形、非線形分類をサポートするだ
けでなく、線形、非線形回帰もサポートします。ポイントは、目的を逆にすることであります。マージン違反を減らしながらふたつのクラスの間にもっとも太い道を通すのではなく、SVM 回帰はマージン違
反を減らしながら道のなかに入るインスタンスができる限り多くなるようにします（この場合のマージン違反は、道に入っていないことである）。

道の太さは、ハイパーパラメータ" によって調節されます。下図は、無作為な線形データに対して訓練したふたつの線形SVM 回帰モデルを示しています。
片方はマージンが大きく（" = 1.5）、もう片方はマージンが小さいモデルです（" = 0.5）。
マージンに入る訓練インスタンスを増やしても、モデルの予測に影響はありません。そのようなことから、このモデルは、ε不感（ε-insensitive）だと言われています。


scikit-learn のLinearSVR クラスを使えば、線形SVM 回帰を行うことができます。次のコードは、の左側のグラフが表すモデルを作ります（まず、訓練データをスケーリングして、中央に移動しなければならない）。

In [0]:
np.random.seed(42)
m = 50
X = 2 * np.random.rand(m, 1)
y = (4 + 3 * X + np.random.randn(m, 1)).ravel()

In [0]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

In [0]:
svm_reg1 = LinearSVR(epsilon=1.5, random_state=42)
svm_reg2 = LinearSVR(epsilon=0.5, random_state=42)
svm_reg1.fit(X, y)
svm_reg2.fit(X, y)

def find_support_vectors(svm_reg, X, y):
    y_pred = svm_reg.predict(X)
    off_margin = (np.abs(y - y_pred) >= svm_reg.epsilon)
    return np.argwhere(off_margin)

svm_reg1.support_ = find_support_vectors(svm_reg1, X, y)
svm_reg2.support_ = find_support_vectors(svm_reg2, X, y)

eps_x1 = 1
eps_y_pred = svm_reg1.predict([[eps_x1]])

In [0]:
def plot_svm_regression(svm_reg, X, y, axes):
    x1s = np.linspace(axes[0], axes[1], 100).reshape(100, 1)
    y_pred = svm_reg.predict(x1s)
    plt.plot(x1s, y_pred, "k-", linewidth=2, label=r"$\hat{y}$")
    plt.plot(x1s, y_pred + svm_reg.epsilon, "k--")
    plt.plot(x1s, y_pred - svm_reg.epsilon, "k--")
    plt.scatter(X[svm_reg.support_], y[svm_reg.support_], s=180, facecolors='#FFAAAA')
    plt.plot(X, y, "bo")
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.legend(loc="upper left", fontsize=18)
    plt.axis(axes)

plt.figure(figsize=(9, 4))
plt.subplot(121)
plot_svm_regression(svm_reg1, X, y, [0, 2, 3, 11])
plt.title(r"$\epsilon = {}$".format(svm_reg1.epsilon), fontsize=18)
plt.ylabel(r"$y$", fontsize=18, rotation=0)
#plt.plot([eps_x1, eps_x1], [eps_y_pred, eps_y_pred - svm_reg1.epsilon], "k-", linewidth=2)
plt.annotate(
        '', xy=(eps_x1, eps_y_pred), xycoords='data',
        xytext=(eps_x1, eps_y_pred - svm_reg1.epsilon),
        textcoords='data', arrowprops={'arrowstyle': '<->', 'linewidth': 1.5}
    )
plt.text(0.91, 5.6, r"$\epsilon$", fontsize=20)
plt.subplot(122)
plot_svm_regression(svm_reg2, X, y, [0, 2, 3, 11])
plt.title(r"$\epsilon = {}$".format(svm_reg2.epsilon), fontsize=18)
save_fig("svm_regression_plot")
plt.show()

非線形回帰には、カーネル化されたSVM モデルを使えばよいです。たとえば、下図は、無作為な二次回帰訓練セットに対する二次多項式カーネルを使ったSVM 回帰を示しています。

左側のグラフはほとんど正則化されていない（つまりC の値が大きい）が、右側のグラフはかなり正則化され
ています（つまりC の値が小さい）。
次のコードは、scikit-learn のSVR クラス（カーネルトリックをサポートする）を使って、図の左側のモデルを作ります。SVR クラスはSVC クラスの回帰版と言うべきもので、同じようにLinearSVR クラスはLinearSVC クラスの回帰版であります。LinearSVR クラスは訓練セット
に対して線形にスケーリングする（LinearSVC クラスと同様に）が、SVR クラスは訓練セットが大きくなるとそれ以上に非常に遅くなります（SVC クラスと同様に）。

In [0]:
np.random.seed(42)
m = 100
X = 2 * np.random.rand(m, 1) - 1
y = (0.2 + 0.1 * X + 0.5 * X**2 + np.random.randn(m, 1)/10).ravel()

In [0]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="auto")
svm_poly_reg.fit(X, y)

In [0]:
from sklearn.svm import SVR

svm_poly_reg1 = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="auto")
svm_poly_reg2 = SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1, gamma="auto")
svm_poly_reg1.fit(X, y)
svm_poly_reg2.fit(X, y)

In [0]:
plt.figure(figsize=(9, 4))
plt.subplot(121)
plot_svm_regression(svm_poly_reg1, X, y, [-1, 1, 0, 1])
plt.title(r"$degree={}, C={}, \epsilon = {}$".format(svm_poly_reg1.degree, svm_poly_reg1.C, svm_poly_reg1.epsilon), fontsize=18)
plt.ylabel(r"$y$", fontsize=18, rotation=0)
plt.subplot(122)
plot_svm_regression(svm_poly_reg2, X, y, [-1, 1, 0, 1])
plt.title(r"$degree={}, C={}, \epsilon = {}$".format(svm_poly_reg2.degree, svm_poly_reg2.C, svm_poly_reg2.epsilon), fontsize=18)
save_fig("svm_with_polynomial_kernel_plot")
plt.show()